In [17]:
# La clase `Model` se hace cargo de los atributos a nivel del modelo, maneja los agentes. 
# Cada modelo puede contener múltiples agentes y todos ellos son instancias de la clase `Agent`.
from mesa import Agent, Model 

# Con `SimultaneousActivation` hacemos que todos los agentes se activen de manera simultanea.
from mesa.time import SimultaneousActivation

# Vamos a hacer uso de `DataCollector` para obtener el grid completo cada paso (o generación) y lo usaremos para graficarlo.
from mesa.datacollection import DataCollector

# mathplotlib lo usamos para graficar/visualizar como evoluciona el autómata celular.
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation as animation
plt.rcParams["animation.html"] = "jshtml"
matplotlib.rcParams['animation.embed_limit'] = 2**128

# Definimos los siguientes paquetes para manejar valores númericos.
import numpy as np
import pandas as pd

# Definimos otros paquetes que vamos a usar para medir el tiempo de ejecución de nuestro algoritmo.
import time
import datetime

TOTAL_MOVES=0

def get_grid(model):
    '''
    Esta es una función auxiliar que nos permite guardar el grid para cada uno de los agentes.
    param model: El modelo del cual optener el grid.
    return una matriz con la información del grid del agente.
    '''
    grid = np.zeros((model.grid.width, model.grid.height))
    for cell in model.grid.coord_iter():
        cell_content, x, y = cell
        for content in cell_content:
            if isinstance(content, Vacuum):
                grid[x][y] = 2
            elif isinstance(content, Floor):
                if content.live == 1:
                    grid[x][y] = 1
                else:
                    grid[x][y] = 0
            else:
                grid[x][y] = 0
    return grid

In [18]:
class RandomWalker(Agent):

    grid = None
    x = None
    y = None
    moore = True

    def __init__(self, unique_id, pos, model, moore=True):
        super().__init__(unique_id, model)
        self.pos = pos
        self.moore = moore

    def random_move(self):
        # Pick the next cell from the adjacent cells.
        next_moves = self.model.grid.get_neighborhood(self.pos, self.moore, True)
        next_move = self.random.choice(next_moves)
        # Now move:
        self.model.grid.move_agent(self, next_move)

In [19]:
class Vacuum(RandomWalker):

    def __init__(self, unique_id, pos, model, moore):
        super().__init__(unique_id, pos, model, moore=moore)
        self.live=2
        #self.moves=0

    def step(self):
        self.random_move()
        #self.moves=self.moves+1
        global TOTAL_MOVES
        TOTAL_MOVES = TOTAL_MOVES + 1
        # Si esta sucio, limpia
        x, y = self.pos
        this_cell = self.model.grid.get_cell_list_contents([self.pos])
        dirt = [obj for obj in this_cell if isinstance(obj, Floor)]
        if len(dirt) > 0:
            dirt_to_eat = self.random.choice(dirt)
            #print(sheep_to_eat.live)

            # Elimina suciedad
            dirt_to_eat.live = 0
            #print(sheep_to_eat.live)

                
class Floor(Agent):
    '''
    Representa a un agente o una celda con estado vivo (1) o muerto (0)
    '''
    def __init__(self, unique_id, pos, model):
        '''
        Crea un agente con estado inicial aleatorio de 0 o 1, también se le asigna un identificador 
        formado por una tupla (x,y). También se define un nuevo estado cuyo valor será definido por las 
        reglas mencionadas arriba.
        '''
        super().__init__(unique_id, model)
        self.live = 1
        self.pos=pos
        self.next_state = None
    
    def step(self):
        this_cell = self.model.grid.get_cell_list_contents([self.pos])

In [20]:
from mesa import Model
from mesa.space import MultiGrid
from mesa.datacollection import DataCollector


class CleaningModel(Model):
    def __init__(self, width, height, initial_dirt, initial_bot):
        #self.num_agents = width * height
        self.initial_dirt=initial_dirt
        self.initial_bot=initial_bot
        self.grid = MultiGrid(width, height, True)
        self.schedule = SimultaneousActivation(self)
        
        
        #spawn piso sucio
        for i in range(self.initial_dirt):
            x = self.random.randrange(width)
            y = self.random.randrange(height)
            dirt = Floor(i,(x, y), self)
            self.grid.place_agent(dirt, (x, y))
            self.schedule.add(dirt)

        #spawn aspiradoras
        for i in range(self.initial_bot):
            bot = Vacuum(i+initial_dirt, (1, 1), self,True)
            self.grid.place_agent(bot, (1, 1))
            self.schedule.add(bot)
        
        # Aquí definimos con colector para obtener el grid completo.
        self.datacollector = DataCollector(
            model_reporters={"Grid": get_grid})
    def isallclean(self):
        filthy_cells = 0
        for cell in model.grid.coord_iter():
            cell_content, x, y = cell
            for content in cell_content:
                if isinstance(content, Floor):
                    if content.live == 1:
                        filthy_cells = filthy_cells + 1
                
        if filthy_cells>=1:
            #print("hay sucio")
            return False
        else:
            #print("no hay sucio")
            return True
        
    def step(self):
        '''
        En cada paso el colector tomará la información que se definió y almacenará el grid para luego graficarlo.
        '''
        self.schedule.step()
        self.datacollector.collect(self)
        

In [21]:
#Datos Iniciales
TIEMPO_MAXIMO_EJECUCION = 0.004 #Tiempo Maximo de Ejecucion
initial_bots            = int(input('Numero inicial de Aspiradoras: '))#6 #Numero de Agentes
initial_dirt_percentage = float(input('Porcentaje inicial de Pisos sucios: '))#.16 #Mugre Inicial (Porcentaje)
width                   = int(input('Width: '))#6 #M
height                  = int(input('Height: '))#6 #N

initial_dirt = round(initial_dirt_percentage*(width*height))


#cont=0
start_time = time.time()
tiempo_inicio=str(datetime.timedelta(seconds=TIEMPO_MAXIMO_EJECUCION))
model = CleaningModel(width, height, initial_dirt, initial_bots)
while((time.time()-start_time) < TIEMPO_MAXIMO_EJECUCION and model.isallclean() == False):
    #print(cont)
    model.step()
    #cont=cont+1

# Imprimimos el tiempo que le tomó correr al modelo.
print('Tiempo de ejecución:', str(datetime.timedelta(seconds=(time.time() - start_time))))



Numero inicial de Aspiradoras: 6
Numero inicial de Pisos sucios: .16
Width: 6
Height: 6
Tiempo de ejecución: 0:00:00.004001


In [22]:
all_grid = model.datacollector.get_model_vars_dataframe()
print(type(all_grid))
print(all_grid)

<class 'pandas.core.frame.DataFrame'>
                                                 Grid
0   [[0.0, 2.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 2....
1   [[0.0, 2.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 2....
2   [[0.0, 2.0, 2.0, 0.0, 0.0, 0.0], [0.0, 2.0, 0....
3   [[0.0, 2.0, 0.0, 0.0, 0.0, 0.0], [0.0, 2.0, 0....
4   [[0.0, 0.0, 2.0, 0.0, 0.0, 0.0], [0.0, 2.0, 0....
5   [[0.0, 0.0, 0.0, 2.0, 0.0, 0.0], [2.0, 0.0, 0....
6   [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 2....
7   [[2.0, 0.0, 0.0, 0.0, 0.0, 2.0], [0.0, 0.0, 0....
8   [[2.0, 0.0, 0.0, 0.0, 0.0, 0.0], [2.0, 2.0, 0....
9   [[0.0, 0.0, 0.0, 0.0, 0.0, 2.0], [0.0, 0.0, 2....
10  [[0.0, 0.0, 0.0, 0.0, 0.0, 2.0], [2.0, 0.0, 2....
11  [[0.0, 2.0, 2.0, 0.0, 0.0, 0.0], [2.0, 0.0, 0....
12  [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 2.0, 0....
13  [[2.0, 0.0, 0.0, 0.0, 0.0, 0.0], [2.0, 2.0, 2....
14  [[0.0, 2.0, 0.0, 0.0, 0.0, 0.0], [0.0, 2.0, 0....


In [23]:
%%capture
fig, axs = plt.subplots(figsize=(7,7))
axs.set_xticks([])
axs.set_yticks([])
patch = plt.imshow(all_grid.iloc[0][0], cmap='Greys')

def animate(i):
    patch.set_data(all_grid.iloc[i][0])
    
anim = animation.FuncAnimation(fig, animate, frames=len(all_grid))

In [24]:
anim

# Porcentaje de celdas limpias contra las totales

In [25]:
last_grid = all_grid.iloc[-1][0].astype(int)
print(last_grid)
clean_cells=0

clean_cells = clean_cells + np.count_nonzero(last_grid == 2)
clean_cells = clean_cells + np.count_nonzero(last_grid == 0)
print(clean_cells)


print((clean_cells*100)/(width*height))

[[0 2 0 0 0 0]
 [0 2 0 0 0 0]
 [0 0 2 0 0 2]
 [0 0 0 1 0 2]
 [1 0 0 0 0 0]
 [0 0 1 0 0 0]]
33
91.66666666666667


# Numero de Movimientos en total

In [26]:
TOTAL_MOVES

90